<a href="https://colab.research.google.com/github/kimdonggyu2008/combined/blob/main/Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
from transformers import T5Tokenizer, T5ForConditionalGeneration
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# T5 모델 및 토크나이저 불러오기
# summarizer_model = T5ForConditionalGeneration.from_pretrained("t5-large").to(device)
# summarizer_tokenizer = T5Tokenizer.from_pretrained("t5-large")


In [ ]:

# def detect_site(url):
#     parsed_url = urlparse(url)
#     domain = parsed_url.netloc

#     if "bbc" in domain:
#         return "bbc"
#     elif "cnn" in domain:
#         return "cnn"
#     else:
#         return None

# def get_filtered_main_content(url):
#     content = ""

#     # HTML 소스 가져오기
#     response = requests.get(url)
#     if response.status_code != 200:
#         return f"Failed to fetch the webpage! URL: {url}"

#     # 인코딩 설정
#     response.encoding = response.apparent_encoding

#     soup = BeautifulSoup(response.content, 'html.parser')

#     # 사이트 감지
#     site = detect_site(url)
#     if site == "bbc":
#         text_blocks = soup.select('div[data-component="text-block"]')
#         for block in text_blocks:
#             content += block.get_text(strip=True) + "\n\n"
#     elif site == "cnn":
#         paragraphs = soup.select("p.paragraph.inline-placeholder.vossi-paragraph")
#         for paragraph in paragraphs:
#             content += paragraph.get_text(strip=True) + "\n\n"
#     else:
#         return None  #지원되지 않는 사이트

#     return content.strip()

# def split_text_with_last_sentence_overlap(text, target_chunk_length=1024):
#     sentences = re.split(r'(?<=[.!?]) +', text)
#     chunks = []
#     current_chunk = ""

#     for sentence in sentences:
#         if len(current_chunk.split()) + len(sentence.split()) <= target_chunk_length:
#             current_chunk += sentence + " "
#         else:
#             chunks.append(current_chunk.strip())
#             current_chunk = sentence + " "

#     if current_chunk:
#         chunks.append(current_chunk.strip())

#     return chunks

# def summarize_long_text(article_text, target_chunk_length=1024):
#     chunks = split_text_with_last_sentence_overlap(article_text, target_chunk_length)
#     summaries = []
#     total_time = 0  # Summarization 소요 시간 기록

#     for chunk in chunks:
#         try:
#             start_time = time.time()
#             inputs = summarizer_tokenizer(chunk, max_length=target_chunk_length, return_tensors="pt", truncation=True).to(device)
#             summary_ids = summarizer_model.generate(
#                 inputs["input_ids"],
#                 max_length=100,
#                 min_length=50,
#                 length_penalty=2.0,
#                 num_beams=2,
#                 early_stopping=True
#             )
#             summary = summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#             summaries.append(summary)
#             end_time = time.time()
#             total_time += end_time - start_time
#         except Exception as e:
#             print(f"Error processing chunk: {chunk[:100]}... | Error: {e}")
#             summaries.append("Summarization failed for this chunk.")

#     print(f"Total summarization time: {total_time:.2f} seconds")
#     return " ".join(summaries)



# def create_summary_dataset(input_file, output_file):
#     results = []

#     with open(input_file, 'r') as file:
#         urls = file.read().split(',')

#     for url in urls:
#         url = url.strip()
#         if not url:
#             continue

#         # URL 처리
#         article = get_filtered_main_content(url)
#         if not article:  # Article이 없으면 스킵
#             continue

#         highlight = summarize_long_text(article)

#         # 정상적으로 처리된 URL만 결과에 추가
#         results.append({"url": url, "article": article, "highlight": highlight})

#     # 결과를 데이터프레임으로 저장
#     summary_df = pd.DataFrame(results)
#     summary_df.to_csv(output_file, index=False, encoding='utf-8-sig')
#     print(f"Summary dataset saved to {output_file}")

# if __name__ == "__main__":
#     summary_input_file = "/content/drive/MyDrive/summarizer/data/test_summary.txt"  # URL 파일 경로
#     summary_output_file = "/content/drive/MyDrive/summarizer/data/crawled_summary.csv"  # 요약 데이터셋 경로
#     create_summary_dataset(summary_input_file, summary_output_file)

In [16]:
# #요약 데이터셋 길이
# import pandas as pd

# # CSV 파일 경로 입력
# csv_file_path = "/content/drive/MyDrive/summarizer/data/crawled_summary.csv"  # 여기에 실제 파일 경로를 입력하세요.

# # CSV 파일 읽기
# data = pd.read_csv(csv_file_path)

# # 데이터셋의 길이 출력
# print(f"Dataset length: {len(data)} rows")

Dataset length: 1163 rows


In [ ]:
import time
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_translation_models():
    translator_model_name = "facebook/nllb-200-3.3B"

    model = AutoModelForSeq2SeqLM.from_pretrained(translator_model_name, forced_bos_token_id=256098).to(device)
    tokenizer = AutoTokenizer.from_pretrained(translator_model_name, src_lang="eng_Latn", tgt_lang="kor_Hang")

    return model, tokenizer

model, tokenizer = load_translation_models()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/6.93G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/8.55G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [ ]:

def translate_text(text, src_lang, tgt_lang):

    start_time = time.time()

    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang

    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)

    outputs = model.generate(
        inputs["input_ids"],
        max_length=512,
        num_beams=4,
        early_stopping=True
    )
    translated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    end_time = time.time()
    print(f"Translation from {src_lang} to {tgt_lang} completed in {end_time - start_time:.2f} seconds.")
    return translated_text

def create_translation_dataset(summary_df, output_file):

    results = []
    total_start_time = time.time()

    for _, row in summary_df.iterrows():
        original_summary = row["highlight"]
        translations = {"en": original_summary}

        target_languages = {
            "fra_Latn": "French",
            "deu_Latn": "German",
            "rus_Cyrl": "Russian",
            "jpn_Jpan": "Japanese",
        }

        for lang_code, lang_name in target_languages.items():
            try:
                # 영어 -> 중간 언어 번역
                intermediate_translation = translate_text(original_summary, "eng_Latn", lang_code)
                # 중간 언어 -> 한국어 번역
                back_translation = translate_text(intermediate_translation, lang_code, "kor_Hang")
                translations["ko"] = back_translation
            except Exception as e:
                print(f"Translation failed for {lang_name}: {e}")
                translations["ko"] = "Translation failed"

            # 결과 추가
            results.append({
                "ko": translations["ko"],  # 역번역된 한국어
                "en": translations["en"]  # 영어 원문
            })

    total_end_time = time.time()
    print(f"Total translation time: {total_end_time - total_start_time:.2f} seconds.")

    translation_df = pd.DataFrame(results)
    translation_df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"Translated dataset saved to {output_file}")

In [6]:
summary_data_file = "/content/drive/MyDrive/summarizer/data/crawled_summary.csv"
translation_output_file = "/content/drive/MyDrive/summarizer/data/translated_summary.csv"
summary_df = pd.read_csv(summary_data_file)
create_translation_dataset(summary_df, translation_output_file)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1493: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Translation from eng_Latn to rus_Cyrl completed in 7.28 seconds.
Translation from rus_Cyrl to kor_Hang completed in 7.39 seconds.
Translation from eng_Latn to jpn_Jpan completed in 7.29 seconds.
Translation from jpn_Jpan to kor_Hang completed in 7.29 seconds.
Translation from eng_Latn to fra_Latn completed in 3.81 seconds.
Translation from fra_Latn to kor_Hang completed in 2.31 seconds.
Translation from eng_Latn to deu_Latn completed in 3.81 seconds.
Translation from deu_Latn to kor_Hang completed in 2.26 seconds.
Translation from eng_Latn to rus_Cyrl completed in 3.81 seconds.
Translation from rus_Cyrl to kor_Hang completed in 3.68 seconds.
Translation from eng_Latn to jpn_Jpan completed in 3.81 seconds.
Translation from jpn_Jpan to kor_Hang completed in 2.26 seconds.
Translation from eng_Latn to fra_Latn completed in 7.97 seconds.
Translation from fra_Latn to kor_Hang completed in 7.65 seconds.
Translation from eng_Latn to deu_Latn completed in 7.9

KeyboardInterrupt: 

In [8]:
print(results)

NameError: name 'results' is not defined

In [10]:
# 번역데이터셋 길이
import pandas as pd

# CSV 파일 경로 입력
csv_file_path = "/content/drive/MyDrive/summarizer/data/translated_summary.csv"  # 여기에 실제 파일 경로를 입력하세요.

# CSV 파일 읽기
data = pd.read_csv(csv_file_path)

# 데이터셋의 길이 출력
print(f"Dataset length: {len(data)} rows")

Dataset length: 200 rows


In [15]:
import pandas as pd

# CSV 파일 경로
csv_file_path = "/content/drive/MyDrive/summarizer/data/translated_summary.csv"  # 실제 파일 경로 입력

# CSV 파일 읽기
data = pd.read_csv(csv_file_path)

# 데이터셋의 길이 출력
print(f"Dataset length: {len(data)} rows")

# 데이터프레임 전체 보기
print("\nFull DataFrame Preview:")
print(data.head())  # 전체 데이터프레임 상위 5개 행 출력

# 각 열별로 데이터 확인
for column in data.columns:
    print(f"\nColumn: {column} (Top 5 Rows)")
    print(data[column].head())  # 각 열의 상위 5개 데이터 출력


Dataset length: 200 rows

Full DataFrame Preview:
                                                  ko  \
0  인스타그램은 사람들에게 권장되는 모든 컨텐츠를 완전히 삭제할 수 있는 새로운 기능을...   
1  인스타그램은 사람들에게 추천하는 모든 컨텐츠를 완전히 삭제할 수 있는 새로운 기능을...   
2  인스타그램은 사람들에게 권장되는 모든 컨텐츠를 완전히 삭제할 수 있는 새로운 기능을...   
3  인스타그램은 사람들에게 추천하는 모든 컨텐츠를 완전히 삭제할 수 있는 새로운 기능을...   
4  TheChatty Bus는 10시부터 14시까지 노리치 버스역에 정차할 예정입니다....   

                                                  en  
0  Instagram is testing a new feature which allow...  
1  Instagram is testing a new feature which allow...  
2  Instagram is testing a new feature which allow...  
3  Instagram is testing a new feature which allow...  
4  TheChatty Bus will be stationed at Norwich Bus...  

Column: ko (Top 5 Rows)
0    인스타그램은 사람들에게 권장되는 모든 컨텐츠를 완전히 삭제할 수 있는 새로운 기능을...
1    인스타그램은 사람들에게 추천하는 모든 컨텐츠를 완전히 삭제할 수 있는 새로운 기능을...
2    인스타그램은 사람들에게 권장되는 모든 컨텐츠를 완전히 삭제할 수 있는 새로운 기능을...
3    인스타그램은 사람들에게 추천하는 모든 컨텐츠를 완전히 삭제할 수 있는 새로운 기능을...
4    TheChatty Bus는 10시부터 14시까지 노리치 버스